# Model Seçimi

Şimdiye kadar elimizdeki veriyi Train-Test olarak ikiye böldük. 

Bu yöntem tek başına, her zaman güvenilir olmayabilir, çünkü Train-Test şeklinde böldüğümüz zaman gerçek veriyi temsil edip etmediğimizden emin değiliz.

Ek olarak küçük veri setlerinde Test datayı modele hiç göstermediğimiz için öğrenme miktarımız azalıyor olacaktır.

## Cross Validation

**Cross Validation**, tek bir Train-Test bölümü yapmak yerine bunu defalarca yaparak, bu problemi çözer.

Her seferinde yeni bir Train-Test verisi olacağı için, gerçek veriyi temsil yeteneği artmış olur ve işimizi daha az şansa bırakmış oluruz.

Birbirinden farklı Cross Validation yöntemleri vardır. 

Biz en fazla kullanılan **K-Fold Cross Validation** ve **Stratified K-Fold Cross Validation** yöntemlerini inceleyeceğiz.

### Cross Validation Nasıl Çalışır

* Veri Setini rasgele olarak karıştırırız
* Veriyi K adet parçaya (fold) böleriz (K=5 veya K=10 gibi)
* Bir tane parçayı (fold) Test için ayırıyoruz ve geri kalan K-1 adet parçayı Train olarak kullanırız
* K-1 parça ile modeli eğitip (fit), ayırdığımız tek parça ile Test ederiz
* Şimdi başka bir parçayı alıp, döngüyü tekrarlarız
* Her iterasyonda farkı bir Train-Test datası almış oluruz böylece
* K adet parçanın her biri için bu işlemi yaptığımız için elimizde K adet Loss sonucu olmuş olacak.
* Bu sonuçların ortalamasını alıp, ortalama Loss değerimizi bulmuş oluruz.

<img src='images/kfold.svg' />

---

<img src='images/KFold_Cross_Validation.png' />

---

<img src='images/cv.jpg' />

Bu şekilde, K kere veriyi Train-Test split ederek hem Overfitting hem de Underfitting sorunundan kaçmış oluruz. 

Çünkü verimizin tümü üzerinde çalışma şansı bulmuş oluruz.

Tek bir Test-Split yapmış olsaydık, o zaman modelimiz hiçbir zaman Test datayı görmüyor olacaktı öğrenirken.

**Önemli Not:**

Eğer elimizdeki veri seti küçükse, direk bütün veri ile Cross Validation yaparız.

Eğer elimizdeki veri seti büyükse, o zaman:
* nihai Test datayı ayırırız
* Train data üzerinden Cross Validation yaparız (yani Train datayı -> Train-Test diye ayırırız)

### Hyperparameter Tuning

Cross Validation'ın (CV) nasıl çalıştığını anladığımıza göre şimdi CV'yi Hyperparameter Tuning için nasıl kullanabileceğimize bakalım.

Hyperparameter Tuning yani en iyi modeli bulmak için modelimizi faklı parmetre kombinasyonları üzerinde CV ile çalışıtırırız. 

Böylece, skorları kaydedip analiz ederiz.

Bunun için scikit-learn'ün **cross_val_score** fonksiyonunu kullanacağız. Bu fonksiyon cross-validation kullanarak bize skorlar üretecek bu skorlara göre biz en iyi parametrelere karar vereceğiz.

### Cross Validation'ın Avantajları:

* Bütün veriyi hem Train hem de Test için kullandığımız için, veri kaybı yaşamayız ve küçük veri setlerinde bu çok önemlidir.
* Çeşitli şekillerde Train-Test datası aldığımız için modelin temsil kapasitesi artar
* Farklı parametreler üzerinde analiz yapıp en iyi parametre kombinasyonunu bulmamıza yardım eder

### Cross Validation Ne Zaman Kullanılamaz:
* Time Series gibi datalarda (sıranın önemli olduğu yerlerde) kullanılamaz.

## Stratified K-Fold Cross Validation

Cross Validation yaparken veriyi sadece rasgele olarak K adet parçaya ayırmak her zaman faydalı olmayabilir.

Toplam veri seti rasgle bölündüğü için verinin dağılımı farkı olacaktır.

Oysa, gerçek veri ile aynı dağılıma sahip K parçaları yaratabilsek çok daha sağlıklı olacaktır.

İşte bunun için **Stratified K-Fold Cross Validation** kullanılır.

Verimiz K adet parçaya ayrılırken parçarladaki ortalama dağılımlar gerçek veriye benzetilir. Böylece parçalarımız bütün veriyi daha iyi temsil eder duruma gelir.

Diyelim ki verimizde iki adet sınıfımız olsun ve dağılımı şu şekilde olsun:
* 0 -> %60
* 1 -> %40

Stratified K-Fold ile diyelim ki K = 5 adet praçaya böldük veriyi.

Bu durumda her bir parçanın içindeki 0-1 sınıfları oranı yaklaşık olarak %60-%40 olacak şekilde bir dağılım olacaktır.

---

## K-Fold Cross Validation - Regression Örneği

Bu örnekte Kaggle'da yer alan örnek bir yarışma datası kullanacak ve K-Fold CV ve Stratified K-Fold CV'yi göreceğiz:

https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data

Kullanacaklarımız:
* numpy
* pandas
* KFold: Sklearn'ün K-Fold Cross Validator Class'ı
* StratifiedKFold: Sklearn'ün Stratified K-Fold Cross Validator Class'ı
* cross_val_score: Sklearn'ün cross validation skoru veren fonksiyonu
* linear_model: Sklearn'ün lineer modellerinden LinearRegression ve LogisticRegression'ı

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression

#### Veriyi yükleyelim

Data'yı pandas kullanarak yükleyeceğiz, kategorik kolonları ve içinde null olan kolonları sileceğiz.

(Amacımız sadece Cross Validation yapmak olduğu için, Data Preprocessing ve Feature Engineering üzerinde durmayacağız.)

In [2]:
train_data = pd.read_csv('data/housing/train.csv')

In [3]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
# eksik verili satırları sil

train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

In [5]:
train_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [6]:
# içinde null değerler olan sütunları sil

train_data.drop(['LotFrontage', 'GarageYrBlt', 'MasVnrArea', 'Alley', 'PoolQC', 'MiscFeature'], 
                 axis=1, inplace=True)

In [7]:
train_data.head()

,Id,MSSubClass,MSZoning,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,3SsnPorch,ScreenPorch,PoolArea,Fence,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,0,0,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,NaN,0,12,2008,WD,Normal,250000


In [8]:
# sonuç değişkenini al

y = train_data.SalePrice

# sonuç değişkenini çıkar
train_data.drop(['SalePrice'], axis=1, inplace=True)

In [9]:
# sadece numerik kolonları seç

numeric_cols = [cname for cname in train_data.columns if train_data[cname].dtype in ['int64', 'float64']]

X = train_data[numeric_cols].copy()

In [10]:
X

,Id,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,8450,7,5,2003,2003,706,0,150,...,548,0,61,0,0,0,0,0,2,2008
1,2,20,9600,6,8,1976,1976,978,0,284,...,460,298,0,0,0,0,0,0,5,2007
2,3,60,11250,7,5,2001,2002,486,0,434,...,608,0,42,0,0,0,0,0,9,2008
3,4,70,9550,7,5,1915,1970,216,0,540,...,642,0,35,272,0,0,0,0,2,2006
4,5,60,14260,8,5,2000,2000,655,0,490,...,836,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,7917,6,5,1999,2000,0,0,953,...,460,0,40,0,0,0,0,0,8,2007
1456,1457,20,13175,6,6,1978,1988,790,163,589,...,500,349,0,0,0,0,0,0,2,2010
1457,1458,70,9042,7,9,1941,2006,275,0,877,...,252,0,60,0,0,0,0,2500,5,2010
1458,1459,20,9717,5,6,1950,1996,49,1029,0,...,240,366,0,112,0,0,0,0,4,2010


In [11]:
print("Input datanın şekli: {} ve sonuç değişkenin şekli: {}".format(X.shape, y.shape))

Input datanın şekli: (1460, 34) ve sonuç değişkenin şekli: (1460,)


Bu işlemlerden sonra datasetimizde 1460 veri (satır) ve 34 değişken (sütun) kaldı.

### KFold ile Model Skoru

**cross_val_score()** kullanacağız ve skorları ondan alacağız.

Lineer Regresyon modeli kullanacağız.

Her bir split için (CV'nin parçaları) bir skor bulacak ve sonra tümü için ortalama skor hesaplayacağız.

Kullanacağımız skor metriği: Root Mean Square Error (RMSE) olacak.

RMSE'yi bulmak için önce Mean Sqaured Error (MSE)'yi bulacağız.

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

In [12]:
# Diyelim K = 5 olsun 
# K -> n_splits

kf = KFold(n_splits = 5, shuffle=True, random_state=42)
kf

KFold(n_splits=5, random_state=42, shuffle=True)

In [13]:
cnt = 1

# split() methodu Train-Test olarak ayırmak için bize indeksleri döner
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt += 1

Fold:1, Train set: 1168, Test set:292
Fold:2, Train set: 1168, Test set:292
Fold:3, Train set: 1168, Test set:292
Fold:4, Train set: 1168, Test set:292
Fold:5, Train set: 1168, Test set:292


In [14]:
# RMSE'yi hesaplamak için - (eksi) ile çarpacağız.
# cross_val_score() dan bize negatif gelecek

def rmse(score):
    rmse = np.sqrt(-score)
    print(f'rmse = {"{:.2f}".format(rmse)}')

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html

In [15]:
# Lineer Regresyon modeli

# score için verdiğimiz değer MSE'nin negatifi, çünkü MSE aslında maliyet demek
# yüksek MSE istemediğimiz için negatif olarak verdik
score = cross_val_score(LinearRegression(), X, y, cv = kf, scoring="neg_mean_squared_error")

In [16]:
print(f'Her bir fold için skor: {score}')

rmse(score.mean())

Her bir fold için skor: [-1.39334669e+09 -1.32533433e+09 -3.39493937e+09 -9.31045536e+08
 -7.16620849e+08]
rmse = 39398.70


In [17]:
np.sqrt(-sum(score) / len(score))

39398.69737756585

## K-Fold Cross Validation - Classification Örneği

Titanic Datasetini kullanacağız.

Çok basit bir preprocessing yapacağız ve kategorik sütunları çıkaracağız.

https://www.kaggle.com/c/titanic/overview

### Verileri yükleyelim

In [18]:
train_data = pd.read_csv('data/titanic/train.csv')

In [19]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [20]:
# eksik verili satırları sil

train_data.dropna(axis=0, subset=['Survived'], inplace=True)

In [21]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [22]:
# sonuç değişkeni

y = train_data.Survived
y

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [23]:
# train datadan y'yi çıkar

train_data.drop(['Survived'], axis=1, inplace=True)

In [24]:
train_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [25]:
# İçinde Null değerler olan Age sütununu sil

train_data.drop(['Age'], axis=1, inplace=True)

In [26]:
train_data.head()

,PassengerId,Pclass,Name,Sex,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,0,0,373450,8.0500,NaN,S


In [27]:
# sadece numerik kolonları seç

numeric_cols = [cname for cname in train_data.columns if train_data[cname].dtype in ['int64', 'float64']]

In [28]:
X = train_data[numeric_cols].copy()

In [29]:
X.head()

,PassengerId,Pclass,SibSp,Parch,Fare
0,1,3,1,0,7.2500
1,2,1,1,0,71.2833
2,3,3,0,0,7.9250
3,4,1,1,0,53.1000
4,5,3,0,0,8.0500


In [30]:
print("Train datanın şekli: {} ve sonuç değişkenin şekli: {}".format(X.shape, y.shape))

Train datanın şekli: (891, 5) ve sonuç değişkenin şekli: (891,)


In [31]:
# ilk 5 train datasını göster

pd.concat([X, y], axis=1).head()

,PassengerId,Pclass,SibSp,Parch,Fare,Survived
0,1,3,1,0,7.2500,0
1,2,1,1,0,71.2833,1
2,3,3,0,0,7.9250,1
3,4,1,1,0,53.1000,1
4,5,3,0,0,8.0500,0


Nihai datamızda 891 adet satır ve 5 adet sütun var. 

Amacımız Survived yani hayatta kalma durumunu 0 veya 1 olarak tahmin etmek. 1 -> hayatta kaldı demek.

### StratifiedKFold ile Model

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

In [32]:
# K = 5
# Stratified K-Fold yaptığımız için foldlardaki sınıf oranları gerçek dataya ile yakın olacaktır.

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [33]:
kf

StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

In [34]:
cnt = 1

# split()  ile indexler
for train_index, test_index in kf.split(X, y):
    print(f'Fold:{cnt}, Train set: {len(train_index)}, Test set:{len(test_index)}')
    cnt+=1

Fold:1, Train set: 712, Test set:179
Fold:2, Train set: 713, Test set:178
Fold:3, Train set: 713, Test set:178
Fold:4, Train set: 713, Test set:178
Fold:5, Train set: 713, Test set:178


**Not:** cross_val_score()'da `cv` parametresini boş geçerseniz default olarak **StratifiedKFold** yapacaktır.

### Logistic Regression Modeli

In [45]:
# CV skorunu alalım
# skorlama yöntemi -> accuracy

score = cross_val_score(LogisticRegression(random_state= 42), X, y, cv = kf, scoring="accuracy")

In [46]:
print(f"Her bir fold'un skoru: {score}")
print(f'Ortalama Skor: {"{:.2f}".format(score.mean())}')

Her bir fold'un skoru: [0.66480447 0.69662921 0.70224719 0.69101124 0.66292135]
Ortalama Skor: 0.68


In [47]:
score.mean()

0.6835226916075576

### Hyperparameter Tuning

Şimdi değişik parametreler deneyerek hangi solver'ın (çözüm algoritması) en iyi olduğunu bulalım:

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [50]:
# Logistic Regression'ın bütün Solver'larını deneylim
solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']


# her bir solver için ortalama score hesaplayalım
# max_iter'i 4000 verdik
for solver in solvers:
    
    score = cross_val_score(LogisticRegression(max_iter=4000, solver=solver, random_state=42), 
                            X, y, cv=kf, scoring="accuracy")
    
    print(f'Ortalama Skor({solver}): {"{:.3f}".format(score.mean())}')


Ortalama Skor(newton-cg): 0.684
Ortalama Skor(lbfgs): 0.684
Ortalama Skor(liblinear): 0.684
Ortalama Skor(sag): 0.678
Ortalama Skor(saga): 0.681


Gördüğünüz gibi en iyi sonuç, 'newton-cg', 'lbfgs', 'liblinear' dan biri çıktı.

**solver{‘newton-cg’, ‘lbfgs’, ‘liblinear’, ‘sag’, ‘saga’}, default=’lbfgs’**